# Consensus network aggregation
- after concat, filter nonsignificant edges with p-values and appropriate threshold of edges ( i filtered with p<0.05 initailly, very roughly)
- count frequency
- meta-analysis using `metap` in R

In [3]:
library(tidyverse)
library(metap)
library(here)

here() starts at /home/jiehoonk/crisprscreen



In [5]:
grns <- list.files(here("tmp/celloracle"), pattern = "*.csv$")
grns

[1] "3182_grn.csv"       "3282_grn.csv"       "3295_grn.csv"       "BT333_grn.csv"      "BT346_grn.csv"     
 [6] "BT363_grn.csv"      "BT364_grn.csv"      "BT368_grn.csv"      "BT389_grn.csv"      "BT390_grn.csv"     
[11] "BT397_grn.csv"      "BT402_grn.csv"      "BT407_grn.csv"      "BT409_grn.csv"      "C3L-02705_grn.csv" 
[16] "C3L-03405_grn.csv"  "C3L-03968_grn.csv"  "C3N-00662_grn.csv"  "C3N-01334_grn.csv"  "C3N-01798_grn.csv" 
[21] "C3N-01814_grn.csv"  "C3N-01815_grn.csv"  "C3N-01816_grn.csv"  "C3N-02181_grn.csv"  "C3N-02188_grn.csv" 
[26] "C3N-02190_grn.csv"  "C3N-02769_grn.csv"  "C3N-02783_grn.csv"  "C3N-02784_grn.csv"  "C3N-03184_grn.csv" 
[31] "C3N-03186_grn.csv"  "C3N-03188_grn.csv"  "G1003_grn.csv"      "G523_grn.csv"       "G549_grn.csv"      
[36] "G566_grn.csv"       "G583_grn.csv"       "G620_grn.csv"       "G637_grn.csv"       "G729_grn.csv"      
[41] "G797_grn.csv"       "G799_grn.csv"       "G837_grn.csv"       "G851_grn.csv"       "G910_grn.csv"      
[46] "G945_grn.csv"       "G946_grn.csv"       "GBM2_grn.csv"       "GBM4_grn.csv"       "GSM4141788_grn.csv"
 [ reached getOption("max.print") -- omitted 132 entries ]